### 'Projects' 폴더 'test' 파일 'text' 열에 데이터 추가

In [1]:
!pip show BeautifulSoup4

Name: beautifulsoup4
Version: 4.9.3
Summary: Screen-scraping library
Home-page: http://www.crummy.com/software/BeautifulSoup/bs4/
Author: Leonard Richardson
Author-email: leonardr@segfault.org
License: MIT
Location: c:\users\dsme\anaconda3\lib\site-packages
Requires: soupsieve
Required-by: conda-build


In [2]:
import pandas as pd
from bs4 import BeautifulSoup
import re
import nltk
import numpy as np
from nltk.corpus import stopwords # 텍스트 데이터 전처리를 위해 별도로 NLTK data를 다운로드 (수동 다운로드 - https://lemontia.tistory.com/802)
stemmer = nltk.stem.PorterStemmer()

In [3]:
train = pd.read_excel('test.xlsx')
train.head()

,text
0,2467호선 M#20A 순회 점검 중 발생된 comment 사항 가피스 제거 후 n...
1,"2469 10H AIR TEST 구간 취부 BKT M.T, 노치 1개 MT(현장 반..."
2,"2469 63B,C,80E damage로 인한 RENEW(TANK TEST 전 RE..."
3,2469 B#405 N2 BOX LEAK 부 가우징 후 M.T\r\n- LEAK부 ...
4,2469호선 진수 전 잔여 곡직 수정작업 요청 건으로 적절한 시점에 조치 요청 드립...


In [4]:
def textmining(file_name, raw_data):
    
    train = pd.read_excel(file_name)
    
    # 1) html 태그 제거
    delete_html = BeautifulSoup(raw_data, 'html.parser').get_text()
    
    # 2) 특수문자 제거
    only_letters = re.sub('[^a-zA-Z가-힣]', ' ', delete_html)
    
    # 3) 모두 소문자로 and 토큰화
    lower_case = only_letters.lower().split()
    
    # 4) 불용어 제거
    stops = set(stopwords.words('english')) # 파이썬에서는 리스트보다 세트로 찾는 게 훨씬 빠르다
    delete_stopwords = [w for w in lower_case if not w in stops]
    
    # 5) 포터스테밍
    words_stemming = [stemmer.stem(w) for w in delete_stopwords]
    
    # 6) 분리되어 있는 리스트를 '공백으로 결합된 문자열'로 반환
    return(' '.join(words_stemming))

In [ ]:
# 전체 데이터 함수에 적용

final_data =[]
for i in range(0, train['text'].size):
    final_data.append(textmining('test.xlsx', train['text'][i]))

In [ ]:
# step 1. CountVectorizer 생성

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline

vectorizer = CountVectorizer(analyzer = 'word', # 문자열 {‘word’, ‘char’, ‘char_wb’} 또는 함수 중 어떤 단위로 토큰화 할지를 정의
                             tokenizer = None, # 함수 또는 None (디폴트), 기본으로 제공하고 있는 방법 외에 방법으로 토큰화 하고자 할 때 함수를 만들어서 넣어줄 수 있음
                             preprocessor = None, # 토큰화 전에 텍스트를 별도로 전처리 할 함수가 있다면 입력
                             stop_words = None, 
                             min_df = 5, # 토큰이 나타날 최소 문서 개수 -> 2번 미만으로 나온 단어(토큰)은 무시
                             ngram_range=(1, 3), # 하나의 토큰에 들어가는 단어의 수 결정 1, 3 -> I, I am, I am tired
                             max_features = 10000 # 생성되는 최대 토큰수(행렬의 열의 수)
                            )
vectorizer

In [ ]:
# step 2. 벡터화 fit 사용

train_data_features = vectorizer.fit_transform(final_data)
vocab = vectorizer.get_feature_names()

In [ ]:
# step 3. 생성된 matrix를 DF로 만들기

dist = np.sum(train_data_features, axis=0) # 각 feature의 총 빈도 확인 (numpy axis 개념 엑셀 참조)
final = pd.DataFrame(dist, columns=vocab) # DF로 만들기

In [ ]:
final.to_excel('Textmining_result.xlsx')
final

In [ ]:
train_data_features.shape